# Analyze Product Sentiment

In [1]:
import turicreate

# Read product review data

In [2]:
products = turicreate.SFrame('amazon_baby.sframe')

# Explore data

In [3]:
products

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


In [4]:
products.groupby('name',operations={'count':turicreate.aggregate.COUNT()}).sort('count',ascending=False)

/Users/junseoklee/my-dir/my-venv/lib/python3.7/site-packages/turicreate/aggregate.py:91: UserWarning: Passing parameter(s) to COUNT is deprecated. This functionality will be removed in the next major release. Any passed parameters are ignored.
  warnings.warn('Passing parameter(s) to COUNT is deprecated. This functionality will be removed in '


name,count
Vulli Sophie the GiraffeTeether ...,785
"Simple Wishes Hands-FreeBreastpump Bra, Pink, ...",562
Infant Optics DXR-5 2.4GHz Digital Video Baby ...,561
Baby Einstein Take AlongTunes ...,547
Cloud b TwilightConstellation Night ...,520
"Fisher-Price BoosterSeat, Blue/Green/Gray ...",489
Fisher-Price RainforestJumperoo ...,450
"Graco Nautilus 3-in-1 CarSeat, Matrix ...",419
Leachco Snoogle TotalBody Pillow ...,388
"Regalo Easy Step WalkThru Gate, White ...",374


# Examine the reivews for the most-reviewed product

In [5]:
giraffe_reviews = products[products['name']=='Vulli Sophie the Giraffe Teether']

In [6]:
giraffe_reviews

name,review,rating
Vulli Sophie the GiraffeTeether ...,He likes chewing on allthe parts especially the ...,5.0
Vulli Sophie the GiraffeTeether ...,My son loves this toy andfits great in the diaper ...,5.0
Vulli Sophie the GiraffeTeether ...,There really should be alarge warning on the ...,1.0
Vulli Sophie the GiraffeTeether ...,All the moms in my moms'group got Sophie for ...,5.0
Vulli Sophie the GiraffeTeether ...,I was a little skepticalon whether Sophie was ...,5.0
Vulli Sophie the GiraffeTeether ...,I have been reading aboutSophie and was going ...,5.0
Vulli Sophie the GiraffeTeether ...,My neice loves her sophieand has spent hours ...,5.0
Vulli Sophie the GiraffeTeether ...,What a friendly face!And those mesmerizing ...,5.0
Vulli Sophie the GiraffeTeether ...,We got this just for myson to chew on instea ...,5.0
Vulli Sophie the GiraffeTeether ...,"My baby seems to likethis toy, but I could ...",3.0


In [7]:
len(giraffe_reviews)

785

In [8]:
giraffe_reviews['rating'].show()

Materializing SArray

# Building a sentiment classifier

## Build word count vectors

In [4]:
products['word_count'] = turicreate.text_analytics.count_words(products['review'])

In [5]:
products

name,review,rating,word_count
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1,'stripping': 1, 'while': ..."
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ..."
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ..."
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ..."
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ..."
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ..."
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ..."
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ..."


In [6]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [39]:
testdic=dict()
testdic.update({"a":0})
testdic

{'a': 0}

In [57]:
# Method 2
def selected_count(word_dic):
    selected_dic=dict()
    for word in selected_words:
        if word in word_dic:
            selected_dic.update({word : word_dic[word]})
    return selected_dic

In [8]:
#not working
def word_count(word,word_dic):
    count = 0
    if word in word_dic:
        count += 1
    return count

In [56]:
#not working
def add_col_selected_words(self):
    for word in selected_words:
        self[word+"_count"] = self['word_count'].apply(word_count,args =(word)) 

In [7]:
# Method 1
for s_word in selected_words:
    def word_count(word_dic):
        count = 0
        if s_word in word_dic:
            count += 1
        return count
    products[s_word] = products["word_count"].apply(word_count)

In [8]:
products

name,review,rating,word_count,awesome,great,fantastic
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0,"{'handles': 1,'stripping': 1, 'while': ...",0,0,0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ...",0,0,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ...",0,1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",0,1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",0,0,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",0,0,0


In [111]:
products=products.remove_columns(s_word+"_count" for s_word in selected_words)

In [65]:
print(type(products))
print(type(products["word_count"]))

<class 'turicreate.data_structures.sframe.SFrame'>
<class 'turicreate.data_structures.sarray.SArray'>


In [79]:
for s_word in selected_words:
    print(s_word+" count is "+str(products[s_word+'_count'].sum()))

awesome count is 3836
great count is 48216
fantastic count is 1694
amazing count is 2548
love count is 35562
horrible count is 1164
bad count is 4570
terrible count is 1188
awful count is 698
wow count is 437
hate count is 1197


In [77]:
products["love"+'_count'].sum()

35562

In [33]:
print("handles" in products['word_count'][0])

True


In [58]:
# Method 2
products['selected_count'] = products['word_count'].apply(selected_count)

In [59]:
products

name,review,rating,word_count,awesome,great,fantastic
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ...",0,0,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ...",0,1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",0,1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",0,0,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1,'recommend': 1, 'high ...",0,0,0


In [67]:
products["selected_count"]

RuntimeError: Cannot perform on non-numeric types!

# Define what is positive and negative sentiment

In [11]:
products['rating'].show()

Materializing SArray...

Done.

Materializing SArray...

Done.

In [9]:
#ignore all 3*  reviews
products = products[products['rating']!= 3]

In [10]:
#positive sentiment = 4-star or 5-star reviews
products['sentiment'] = products['rating'] >= 4

In [11]:
products

name,review,rating,word_count,awesome,great,fantastic
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'recommend': 1, 'moist':1, 'osocozy': 1, 'keps': ...",0,0,0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'quilt': 1,'comfortable': 1, 'bed': ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'tool': 1, 'clever': 1,'little': 1, 'loves': 1, ...",0,0,0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'rock': 1, 'many': 1,'headaches': 1, 'soo' ...",0,1,0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'thumb': 1, 'or': 1,'break': 1, 'trying': 1, ...",0,1,0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'for': 1, 'barnes': 1,'at': 1, 'is': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'right': 1, 'because':1, 'questions': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'like': 1, 'and': 1,'changes': 1, 'the': 1, ...",0,0,1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'in': 1, 'pages': 1,'out': 1, 'run': 1, ...",0,0,0
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'tracker': 1,'recommend': 1, 'high ...",0,0,0


In [12]:
products['sentiment'].show()

Materializing SArray

# Train our sentiment classifier

In [61]:
train_data,test_data = products.random_split(.8,seed=0)

In [62]:
sentiment_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features=['word_count'], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 2         | 11       | 2.669978  | 2.262276     | 0.938650          | 0.916466            |

| 3         | 12       | 2.669978  | 2.576847     | 0.927417          | 0.901814            |

| 4         | 14       | 1.326072  | 3.080100     | 0.945200          | 0.918088            |

| 9         | 20       | 1.326072  | 4.706654     | 0.977654          | 0.917698            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

In [14]:
selected_words_model = turicreate.logistic_classifier.create(train_data,target='sentiment', features = selected_words, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 1.362888     | 0.847341          | 0.846055            |

| 5         | 6        | 1.437890     | 0.847341          | 0.846055            |

| 6         | 7        | 1.514520     | 0.847341          | 0.846055            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [63]:
selected_words_model2 = turicreate.logistic_classifier.create(train_data,target='sentiment', features = ["selected_count"], validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 1

...

| 4         | 5        | 0.465516     | 0.847708          | 0.846385            |

| 5         | 6        | 0.553792     | 0.847708          | 0.846385            |

| 6         | 7        | 0.649678     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [15]:
selected_words_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 12
Number of examples             : 133448
Number of classes              : 2
Number of feature columns      : 11
Number of unpacked features    : 11

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 1.5289

Settings
--------
Log-likelihood                 : 52878.5814

Highest Positive Coefficients
-----------------------------
love                           : 1.6204
(intercept)                    : 1.3234
awesome                        : 1.1992
amazing                        : 1.1611
great                          : 1.0459

Lowest Negative Coefficients
----------------------------
terrible                       : -2.3715
ho

In [64]:
selected_words_model2

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 12
Number of examples             : 133448
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 11

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 6
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.6752

Settings
--------
Log-likelihood                 : 52926.808

Highest Positive Coefficients
-----------------------------
selected_count[love]           : 1.3593
(intercept)                    : 1.3366
selected_count[awesome]        : 1.1335
selected_count[amazing]        : 1.1001
selected_count[fantastic]      : 0.8858

Lowest Negative Coefficients
----------------------------
selected_count[horrible]       : -2.2513
sele

In [16]:
selected_words_model.evaluate(test_data)

{'accuracy': 0.8460545279846264,
 'auc': 0.6913720514731203,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  334  |
 |      0       |        1        |  4994 |
 |      1       |        1        | 27843 |
 |      1       |        0        |  133  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9156923684080707,
 'log_loss': 0.39640266029559057,
 'precision': 0.847915461217529,
 'recall': 0.9952459250786388,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+-----+-----+-------+------+
 | threshold | fpr | tpr |   p   |  n   |
 +-----------+-----+-----+-------+------+
 |    0.0    | 1.0 | 1.0 | 27976 | 5328 |
 |   1e-05   | 1.0 | 1.0 | 27976 | 5328 |
 |   2e-05   |

In [35]:
selected_words_model.coefficients.sort('value',ascending=False).print_rows(12)

+-------------+-------+-------+-----------------------+----------------------+
|     name    | index | class |         value         |        stderr        |
+-------------+-------+-------+-----------------------+----------------------+
|     love    |  None |   1   |    1.62042201302324   | 0.03130114149706122  |
| (intercept) |  None |   1   |   1.3233533108812592  | 0.00898726907144809  |
|   awesome   |  None |   1   |   1.1991933932881274  |  0.0864723335047121  |
|   amazing   |  None |   1   |   1.1610970272710954  |  0.102445833490004   |
|    great    |  None |   1   |   1.045920933549193   | 0.022134924065827115 |
|  fantastic  |  None |   1   |   0.9577875893062683  | 0.11685727648628608  |
|     wow     |  None |   1   | -0.036538032970789835 |  0.1667950424401623  |
|     bad     |  None |   1   |  -1.1098034159247223  |  0.0417399624461776  |
|     hate    |  None |   1   |  -1.4314313653761959  | 0.08096020000806729  |
|    awful    |  None |   1   |  -2.1244236214814096

In [65]:
selected_words_model2.evaluate(test_data)

{'accuracy': 0.8463848186404036,
 'auc': 0.6936022046674926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 |      1       |        0        |  159  |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.3962265467087378,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 532

In [77]:
test_data[test_data['sentiment']==1].shape

(27976, 17)

In [78]:
test_data[test_data['sentiment']==0].shape

(5328, 17)

In [79]:
27976/(27976+5328)

0.8400192169108815

# Apply the sentiment classifier to better understand the Giraffe reviews

In [18]:
products['predicted_sentiment'] = sentiment_model.predict(products, output_type = 'probability')

In [19]:
products

name,review,rating,word_count,sentiment
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0,"{'and': 3, 'love': 1,'it': 2, 'highly': 1, ...",1
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0,"{'and': 2, 'quilt': 1,'it': 1, 'comfortable': ...",1
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0,"{'ingenious': 1, 'and':3, 'love': 2, ...",1
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0,"{'and': 2, 'parents!!':1, 'all': 2, 'puppet.': ...",1
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0,"{'and': 2, 'this': 2,'her': 1, 'help': 2, ...",1
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0,"{'shop': 1, 'noble': 1,'is': 1, 'it': 1, 'as': ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0,"{'and': 2, 'all': 1,'right': 1, 'when': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0,"{'and': 1, 'help': 1,'give': 1, 'is': 1, ' ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0,"{'journal.': 1, 'nanny':1, 'standarad': 1, ...",1
"Baby Tracker&reg; - DailyChildcare Journal, ...",I love this journal andour nanny uses it ...,4.0,"{'all': 1, 'forget': 1,'just': 1, 'food': 1, ...",1


In [66]:
diaper_champ_reviews = products[products['name']== 'Baby Trend Diaper Champ']

In [67]:
diaper_champ_reviews

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,Ok - newsflash. Diapersare just smelly. We've ...,4.0,"{'convenient': 1, 'can':1, 'you': 1, 'fact': 1, ...",0,0,0
Baby Trend Diaper Champ,"My husband and I selectedthe Diaper ""Champ"" ma ...",1.0,"{'try': 1, 'going': 1,'re': 1, 'leaves': 1, ...",0,0,0
Baby Trend Diaper Champ,Excellent diaper disposalunit. I used it in ...,5.0,"{'nose': 1, 'for': 2,'investment': 1, 'wise': ...",0,0,0
Baby Trend Diaper Champ,We love our diaper champ.It is very easy to use ...,5.0,"{'out': 1, 'pull': 1,'open': 1, 'pail': 1, ...",0,0,0
Baby Trend Diaper Champ,Two girlfriends and twofamily members put me ...,5.0,"{'winter': 1,'especially': 1, ...",0,0,0
Baby Trend Diaper Champ,I waited to review thisuntil I saw how it ...,4.0,"{'mom': 1, 'huge': 1,'special': 1, 'good': 1, ...",0,0,0
Baby Trend Diaper Champ,I have had a diaper geniefor almost 4 years since ...,1.0,"{'clean': 1, 'it': 8,'thing': 1, 'years': 1, ...",0,0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1, 'worth': 1,'other': 2, 'would': 1, ...",0,0,0
Baby Trend Diaper Champ,I am so glad I got theDiaper Champ instead of ...,5.0,"{'best': 1, 'that': 1,'handle': 1, 'flip': 1, ...",0,0,0
Baby Trend Diaper Champ,We had 2 diaper Genie'sboth given to us as a ...,4.0,"{'need': 1, 'garbage': 1,'regular': 1, 'part': 1, ...",0,0,0


In [71]:
diaper_champ_reviews['predicted_sentiment'] = sentiment_model.predict(diaper_champ_reviews, output_type = 'probability')

In [68]:
diaper_champ_reviews['predicted_sentiment_selected'] = selected_words_model2.predict(diaper_champ_reviews, output_type = 'probability')

# Sort the Giraffe reviews according to predicted sentiment

In [73]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment_selected', ascending=False)

In [75]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [76]:
diaper_champ_reviews

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,I read a review belowthat can explain exactly ...,4.0,"{'key': 1, 'have': 1,'pieces': 1, 'between': ...",0,0,0
Baby Trend Diaper Champ,I have never written areview for Amazon but I ...,5.0,"{'priceless': 1,'parent': 1, 'makes': 1, ...",0,0,0
Baby Trend Diaper Champ,I originally put thisitem on my baby registry ...,5.0,"{'price': 1, 'worth': 1,'other': 2, 'would': 1, ...",0,0,0
Baby Trend Diaper Champ,Baby Luke can turn aclean diaper to a dirty ...,5.0,"{'around': 1, 'bulk': 1,'any': 1, 't': 1, 'we ...",0,1,0
Baby Trend Diaper Champ,Diaper Champ or DiaperGenie? That was my ...,5.0,"{'be': 1, 'won': 1,'open': 1, 'if': 1, ...",0,1,0
Baby Trend Diaper Champ,I am one of those super-critical shoppers who ...,5.0,"{'helps': 1, 'slower': 1,'changings': 1, 'desi ...",0,0,0
Baby Trend Diaper Champ,I LOOOVE this diaperpail! Its the easies ...,5.0,"{'buy': 1, 'looking': 1,'product': 1, ...",0,0,0
Baby Trend Diaper Champ,"As a first time mother, Iwanted to get the best ...",5.0,"{'ll': 1, 'baby': 1,'recommended': 1, 've': ...",0,0,0
Baby Trend Diaper Champ,I see that there arecomplaints of stinkiness ...,5.0,"{'very': 1, 'seems': 1,'all': 1, 'inside': 1, ...",0,0,0
Baby Trend Diaper Champ,I have a 10 year olddaughter and an 8 month ...,5.0,"{'sorry': 1, 'be': 1,'sell': 1, 'not': 2, ...",0,0,0


In [24]:
giraffe_reviews.tail()

name,review,rating,word_count,sentiment
Vulli Sophie the GiraffeTeether ...,I bought it for my 8month old (I know it's ...,2.0,"{'and': 5, 'enjoy': 1,'right': 1, 'old': 1, ...",0
Vulli Sophie the GiraffeTeether ...,When we first got it Ithought did my ...,2.0,"{'and': 2, 'cute': 1,'all': 1, 'because': 1, ...",0
Vulli Sophie the GiraffeTeether ...,After getting this for mybaby and after she su ...,1.0,"{'just': 1, 'being': 1,'over': 1, 'chalked': 1, ...",0
Vulli Sophie the GiraffeTeether ...,"My daughter loves Sophie,but we received her as a ...",2.0,"{'and': 2, 'the': 3,""don't"": 1, 'doing': 1, ...",0
Vulli Sophie the GiraffeTeether ...,"""Sophie the Giraffe"" hastested positive for ...",1.0,"{'saying': 1,'nitrosamines.': 1, ...",0
Vulli Sophie the GiraffeTeether ...,I received this in a boxset as a gift. It came ...,2.0,"{'cute': 1, 'unhooked':1, 'being': 2, 'stora ...",0
Vulli Sophie the GiraffeTeether ...,I was intrigued with theidea of Sophie the ...,2.0,"{'baby.)': 1, 'all': 2,'able': 1, 'month': 1, ...",0
Vulli Sophie the GiraffeTeether ...,I wanted to love thisproduct and was excited ...,1.0,"{'just': 1, 'may,': 1,'children': 1, 'bummed': ...",0
Vulli Sophie the GiraffeTeether ...,This children's toy isnostalgic and very cute. ...,1.0,"{'expectations.': 1,'refund': 1, 'there': 1, ...",0
Vulli Sophie the GiraffeTeether ...,"My son (now 2.5) LOVEDhis Sophie, and I bought ...",1.0,"{'all': 1, 'just': 1,""don't"": 1, 'over': 2, ...",0


## Show the most positive reviews

In [25]:
giraffe_reviews[0]['review']

"Sophie, oh Sophie, your time has come. My granddaughter, Violet is 5 months old and starting to teeth. What joy little Sophie brings to Violet. Sophie is made of a very pliable rubber that is sturdy but not tough. It is quite easy for Violet to twist Sophie into unheard of positions to get Sophie into her mouth. The little nose and hooves fit perfectly into small mouths, and the drooling has purpose. The paint on Sophie is food quality.Sophie was born in 1961 in France. The maker had wondered why there was nothing available for babies and made Sophie from the finest rubber, phthalate-free on St Sophie's Day, thus the name was born. Since that time millions of Sophie's populate the world. She is soft and for babies little hands easy to grasp. Violet especially loves the bumpy head and horns of Sophie. Sophie has a long neck that easy to grasp and twist. She has lovely, sizable spots that attract Violet's attention. Sophie has happy little squeaks that bring squeals of delight from Viol

In [26]:
giraffe_reviews[1]['review']

"I'm not sure why Sophie is such a hit with the little ones, but my 7 month old baby girl is one of her adoring fans.  The rubber is softer and more pleasant to handle, and my daughter has enjoyed chewing on her legs and the nubs on her head even before she started teething.  She also loves the squeak that Sophie makes when you squeeze her.  Not sure what it is but if Sophie is amongst a pile of her other toys, my daughter will more often than not reach for Sophie.  And I have the peace of mind of knowing that only edible and safe paints and materials have been used to make Sophie, as opposed to Bright Starts and other baby toys made in China.  Now that the research is out on phthalates and other toxic substances in baby toys, I think it's more important than ever to find good quality toys that are also safe for our babies to handle and put in their mouths.  Sophie is a must-have for every new mom in my opinion.  Even if your kid is one of the few that can take or leave her, it's worth

# Most negative reivews

In [27]:
giraffe_reviews[-1]['review']

"My son (now 2.5) LOVED his Sophie, and I bought one for every baby shower I've gone to. Now, my daughter (6 months) just today nearly choked on it and I will never give it to her again. Had I not been within hearing range it could have been fatal. The strange sound she was making caught my attention and when I went to her and found the front curved leg shoved well down her throat and her face a purply/blue I panicked. I pulled it out and she vomited all over the carpet before screaming her head off. I can't believe how my opinion of this toy has changed from a must-have to a must-not-use. Please don't disregard any of the choking hazard comments, they are not over exaggerated!"

In [ ]:
giraffe_reviews[-2]['review']